### Evaluating performance at different training iterations.

We will calculate mAP scores of the trained weights.

In [ ]:
#Clone Darknet Repo to Environment
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 13733, done.
remote: Total 13733 (delta 0), reused 0 (delta 0), pack-reused 13733
Receiving objects: 100% (13733/13733), 12.31 MiB | 15.54 MiB/s, done.
Resolving deltas: 100% (9373/9373), done.


In [ ]:
#Edit Makefile to have GPU, OpenCV Enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
#Verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
#Make Darknet (build)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variable ‘it_tb_res’ [-W

In [ ]:
#Mount Google Drive
%cd ..
from google.colab import drive
drive.mount('/content/drive')

/content
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#give google drive folder alias so we don't have to worry about space in file path
!ln -s /content/drive/My\ Drive/ /mydrive
!ls /mydrive/capstone_2

backup	       darknet53.conv.74  yolo_bdd_custom.cfg
bdd_train.zip  docs		  yolov3.weights


In [ ]:
%pwd

'/content'

In [ ]:
%cd darknet/

/content/darknet


In [ ]:
#copy zip of images
!cp /mydrive/capstone_2/bdd_train.zip ../

In [ ]:
%pwd

'/content/darknet'

In [ ]:
%cd ..

/content


In [ ]:
#Unzip 100k.zip file. Contents should be in darknet/data/bdd_train
!unzip ../content/bdd_train.zip -d darknet/data/

Streaming output truncated to the last 5000 lines.
  inflating: darknet/data/__MACOSX/bdd_train/val/._b6a5e5f0-73e9525b.jpg  
  inflating: darknet/data/bdd_train/val/c993615f-33101e6b.txt  
  inflating: darknet/data/bdd_train/val/b433534e-ff3646aa.txt  
  inflating: darknet/data/bdd_train/val/b4f4bd95-a097514a.jpg  
  inflating: darknet/data/__MACOSX/bdd_train/val/._b4f4bd95-a097514a.jpg  
  inflating: darknet/data/bdd_train/val/c734eb0a-125bba62.txt  
  inflating: darknet/data/bdd_train/val/bf9e3b67-49c1ef1c.txt  
  inflating: darknet/data/bdd_train/val/bdaeda37-a0d637e0.jpg  
  inflating: darknet/data/__MACOSX/bdd_train/val/._bdaeda37-a0d637e0.jpg  
  inflating: darknet/data/bdd_train/val/b1cebfb7-284f5117.jpg  
  inflating: darknet/data/__MACOSX/bdd_train/val/._b1cebfb7-284f5117.jpg  
  inflating: darknet/data/bdd_train/val/b7fd3f86-5366a23c.txt  
  inflating: darknet/data/bdd_train/val/b714a088-e91ac24f.txt  
  inflating: darknet/data/bdd_train/val/bcaf73c1-7be6ff05.txt  
  inflati

In [ ]:
%pwd

'/content'

In [ ]:
%cd darknet/

/content/darknet


In [ ]:
path_images = 'data/bdd_train/'
path_training = 'data/bdd_train/train/'

#create .names file
#we have 10 classes : bike, bus, car, motor, person, rider, traffic light, traffice sign, train, truck

c = 0

with open('classes.names', 'w') as names, open(path_training + 'classes.txt', 'r') as txt:
  for line in txt:
    names.write(line)

    c+=1

In [ ]:
print(c)

10


In [ ]:
#create .data file
with open(path_images + 'bdd.data', 'w') as data:
  data.write('classes = ' + str(c) + '\n')
  data.write('train = ' + path_images + 'train.txt' + '\n')
  data.write('valid = ' + path_images + 'test.txt' + '\n')
  data.write('names = classes.names' + '\n')
  data.write('backup = /mydrive/capstone_2/backup')
  

In [ ]:
%pwd

'/content/darknet'

In [ ]:
#create train.txt and test.txt files

import os

path_train = 'data/bdd_train/train'

os.chdir(path_train)

p = []

for current_dir, dirs, files in os.walk('.'):
  for f in files:
    if f.endswith('.jpg'):
      path_to_save = path_train + f
      p.append(path_to_save + '\n')

print('There are ' + str(len(p)) + ' images in our training set.')

There are 70000 images in our training set.


In [ ]:
%pwd

'/content/darknet/data/bdd_train/train'

In [ ]:
%cd ../

/content/darknet/data/bdd_train


In [ ]:
#write train.txt
with open('train.txt', 'w') as train:
  for e in p:
    train.write(e)

In [ ]:
%pwd

'/content/darknet/data/bdd_train'

In [ ]:
%cd ../../

/content/darknet


In [ ]:
#write test.txt

import os

path_val = 'data/bdd_train/val/'

os.chdir(path_val)
p_test = []

for current_dir, dirs, files in os.walk('.'):
  for f in files:
    if f.endswith('.jpg'):
      path_to_save = path_val + f
      p_test.append(path_to_save + '\n')

print('There are ' + str(len(p_test)) + ' images in our test set.')

There are 10000 images in our test set.


In [ ]:
%pwd

'/content/darknet/data/bdd_train/val'

In [ ]:
%cd ../

/content/darknet/data/bdd_train


In [ ]:
#write test.txt
with open('test.txt','w') as test:
  for e in p_test:
    test.write(e)

In [ ]:
%cd ../../

/content/darknet


In [ ]:
#cope yolo_bdd_custom.cfg and yolo_bdd_8000.weights to VM from Google Drive
!cp /mydrive/capstone_2/yolo_bdd_custom.cfg ./cfg

In [ ]:
%cd ../
!cp /mydrive/capstone_2/backup/yolo_bdd_custom_8000.weights ./darknet

/content


In [ ]:
%cd darknet/

/content/darknet


In [ ]:
#Calculate mAP score for 8000 iterations
!./darknet detector map data/bdd_train/bdd.data cfg/yolo_bdd_custom.cfg yolo_bdd_custom_8000.weights

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 

The mAP on our testing set is 33.19% and that is counting a 0.00% on the train class. This was expected as there were only 136 instances of the train class in the training dataset. It is not surprising that car is the best peforming class as it had by far the largest sample size.